In [1]:
import main_module as m
import zoho_filter_module as fil
import pandas as pd
import re
import numpy as np
import datetime

%load_ext autoreload
%autoreload 2


import gspread
import pandas as pd
from fractions import Fraction

# Pastikan permision access di menu "Share" di rubah ke anyone with the link terlebih dahulu, agar bisa mengunakan cara OAuth
# Jika Ingin lebih dibatasi menggunakan cara google service account lebih disarankan

gc = gspread.oauth(
    credentials_filename= r'C:\Users\fajar.fatoni\Documents\Python\Data\Google Credentials\karma-oauth.json'
)

SHEET_ID = '1ZmyQRySSr6TTqEzOFdhf_U6VEaYwG3Uolz_nwJBoWN8'
SHEET_NAME = 'KSH'


In [2]:
# Open the spreadsheet
spreadsheet_source = gc.open_by_key(SHEET_ID)

# Get all sheet names
sheet_names = [sheet.title for sheet in spreadsheet_source.worksheets()]
sheet_names

# get data gsheet

spreadsheet = gc.open_by_key(SHEET_ID)
worksheet = spreadsheet.worksheet(SHEET_NAME)
rows = worksheet.get_all_records()

# print(rows[:5])

# print('==============================')
# df = pd.DataFrame(rows, dtype="string")
zoho_df = pd.DataFrame(rows)
# df['Mobile Number'] = df['Mobile Number'].apply(Fraction)
zoho_df

# mengisi kolom <NA> dengan value string 'empty' supaya mudah mengecek data member dan booking dengan isin
zoho_df.fillna('empty', inplace=True)

# zoho_df['Departure_Date'] = zoho_df['Departure_Date'].apply(reorder_date_from_yyyy)
count_filter_awal = zoho_df.shape[0]
zoho_df

,Resort Name,Departure_Date,Month Year,Booking Id,,Booker Title,Booker First Name,Booker Last Name,Email,TelNo,Concat_Tel_Mob,Media_Group,Source,MarketingOptIn,Market_Segment,Arrival Status,Contact Type
0,Karma Salford Hall,29/12/2024,Desember 2024,KSH019860,,Mrs,Alex And,Jack,alexandjack@gmail.com,,,GROUP,,Yes,COMP,Arrival Completed,Email and Phone
1,Karma Salford Hall,26/12/2024,Desember 2024,KSH020971,,Mrs,Erika,Goodman,egoodman96@gmail.com,66988930191,7973656599,RETAIL,,Yes,OTA,Arrival Completed,Email and Phone
2,Karma Salford Hall,18/12/2024,Desember 2024,KSH020039,,Mr,John,Haw,john.haw@fidelity-energy.co.uk,447747635540,447747635540,RETAIL,,Yes,DIR,Arrival Completed,Email and Phone
3,Karma Salford Hall,18/12/2024,Desember 2024,KSH020031,,Mr,Steven,Hawes,steveh@salomeurope.com,7918800410,7918800410,DIGITAL,,Yes,DIR,Arrival Completed,Email and Phone
4,Karma Salford Hall,11/12/2024,Desember 2024,KSH021368,,Mr,Tim,Brent,brenttjl@gmail.com,447789768665,447789768665,RETAIL,,Yes,OTA,Arrival Completed,Email and Phone
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
543,Karma Salford Hall,01/06/2025,Juni 2025,,,,Dianna-lee,,dianna.duncan@hotmail.com,61435938676,,,,Yes,EVENT,Arrival Completed,Email and Phone
544,Karma Salford Hall,01/06/2025,Juni 2025,,,,Rebecca,Carter,becky.smart.bs@googlemail.com,447969059458,,,,Yes,EVENT,Arrival Completed,Email and Phone
545,Karma Salford Hall,01/06/2025,Juni 2025,,,,Dilesh,parmar,Dilesh.parmar@live.co.uk,447858224443,,,,Yes,EVENT,Arrival Completed,Email and Phone
546,Karma Salford Hall,01/06/2025,Juni 2025,,,,Louis,Radtke,lousterradtke@gmail.com,447796148410,,,,Yes,EVENT,Arrival Completed,Email and Phone


In [3]:
path = r'G:\.shortcut-targets-by-id\0B_d3smBjzo04eXpvS1NXTWF2RVE\_BOOKING\Booking By Year\FlyBuy Booking Data (BGAU & BGID & KE).xlsx'
df_booking = pd.read_excel(path, dtype='string', sheet_name='BOOKING_BGAU_BGID_KE')

path = r'G:\.shortcut-targets-by-id\0B_d3smBjzo04UUNBME5GOVhoMjA\_ODYSSEY\Phone Members.xlsx'
df_phone = pd.read_excel(path, dtype='string', sheet_name='Sheet1')

path = r'G:\.shortcut-targets-by-id\0B_d3smBjzo04UUNBME5GOVhoMjA\_ODYSSEY\Email Members.xlsx'
df_email = pd.read_excel(path, dtype='string', sheet_name='Sheet1')


# ====================== Awal Filter Umum ====================== 

In [ ]:
# data phone, mobile dam email member yang ada di master data (wash member)

m.clean_number(zoho_df, 'Phone')
m.clean_number(zoho_df, 'Mobile')
m.replace_not_valid_to_empty(zoho_df, 'Phone')
m.replace_not_valid_to_empty(zoho_df, 'Mobile')
zoho_df['Email'] = m.lowercase(zoho_df, 'Email')
m.clean_space(zoho_df, 'Email')
m.clean_not_valid_email_format(zoho_df, 'Email')

m.clean_number(df_booking, 'Phone1')
m.clean_number(df_booking, 'Mobile')
df_booking['Email'] = m.lowercase(df_booking, 'Email')
m.clean_space(df_booking, 'Email')
m.clean_not_valid_email_format(df_booking, 'Email')

m.clean_number(df_phone, 'Mobile')

df_email['Email'] = m.lowercase(df_email, 'Email')
m.clean_space(df_email, 'Email')
m.clean_not_valid_email_format(df_email, 'Email')

c:\Users\fajar.fatoni\Documents\Python\Task Offline\zoho_data_app\main_module.py:104: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[df_columns].replace(' ','', regex=True, inplace=True)
c:\Users\fajar.fatoni\Documents\Python\Task Offline\zoho_data_app\main_module.py:105: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are set

'Email'

In [ ]:
zoho_df.loc[zoho_df['Phone'] == zoho_df['Mobile'],'Mobile'] = 'empty'
zoho_df.loc[
    (zoho_df['Phone'] == 'empty') & (zoho_df['Mobile'] != 'empty'), 
    'Phone'] = zoho_df['Mobile']

In [5]:
count_deleted_wrong_format = m.delete_email_wrong_format(zoho_df, 'Email')
count_after_deleted_wrong_format = zoho_df.shape[0]

# ====================== Akhir Filter Umum ====================== 

In [ ]:
# hitung jumlah duplicate
count_duplicate_general = zoho_df.loc[zoho_df.duplicated(subset=['Phone','Mobile','Email'])].shape[0]

zoho_df.loc[zoho_df.duplicated(subset=['Phone','Mobile','Email'])]

,Resort Name,Departure_Date,Month Year,Booking Id,,Booker Title,Booker First Name,Booker Last Name,Email,TelNo,Concat_Tel_Mob,Media_Group,Source,MarketingOptIn,Market_Segment,Arrival Status,Contact Type


In [ ]:
zoho_df.drop_duplicates(subset=['Phone','Mobile','Email'], inplace=True)
zoho_df.reset_index(drop=True, inplace=True)

# hitung jumlah setelah value duplicate dihapus
count_after_dedup_duplicate = zoho_df.shape[0]

zoho_df

,Resort Name,Departure_Date,Month Year,Booking Id,,Booker Title,Booker First Name,Booker Last Name,Email,TelNo,Concat_Tel_Mob,Media_Group,Source,MarketingOptIn,Market_Segment,Arrival Status,Contact Type
0,Karma Salford Hall,29/12/2024,Desember 2024,KSH019860,,Mrs,Alex And,Jack,alexandjack@gmail.com,empty,empty,GROUP,,Yes,COMP,Arrival Completed,Email and Phone
1,Karma Salford Hall,26/12/2024,Desember 2024,KSH020971,,Mrs,Erika,Goodman,egoodman96@gmail.com,66988930191,7973656599,RETAIL,,Yes,OTA,Arrival Completed,Email and Phone
2,Karma Salford Hall,18/12/2024,Desember 2024,KSH020039,,Mr,John,Haw,john.haw@fidelity-energy.co.uk,447747635540,447747635540,RETAIL,,Yes,DIR,Arrival Completed,Email and Phone
3,Karma Salford Hall,18/12/2024,Desember 2024,KSH020031,,Mr,Steven,Hawes,steveh@salomeurope.com,7918800410,7918800410,DIGITAL,,Yes,DIR,Arrival Completed,Email and Phone
4,Karma Salford Hall,11/12/2024,Desember 2024,KSH021368,,Mr,Tim,Brent,brenttjl@gmail.com,447789768665,447789768665,RETAIL,,Yes,OTA,Arrival Completed,Email and Phone
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
536,Karma Salford Hall,01/06/2025,Juni 2025,,,,Dianna-lee,,dianna.duncan@hotmail.com,61435938676,empty,,,Yes,EVENT,Arrival Completed,Email and Phone
537,Karma Salford Hall,01/06/2025,Juni 2025,,,,Rebecca,Carter,becky.smart.bs@googlemail.com,447969059458,empty,,,Yes,EVENT,Arrival Completed,Email and Phone
538,Karma Salford Hall,01/06/2025,Juni 2025,,,,Dilesh,parmar,dilesh.parmar@live.co.uk,447858224443,empty,,,Yes,EVENT,Arrival Completed,Email and Phone
539,Karma Salford Hall,01/06/2025,Juni 2025,,,,Louis,Radtke,lousterradtke@gmail.com,447796148410,empty,,,Yes,EVENT,Arrival Completed,Email and Phone


In [ ]:
# Cek leads test
test_leads = zoho_df.loc[
    
        zoho_df['Email'].str.contains('^test@', flags=re.I, regex=True) |
        zoho_df['Email'].str.contains('\+test', flags=re.I, regex=True) |
    
        zoho_df['First Name'].str.contains('^test$', flags=re.I, regex=True) |
        zoho_df['Last Name'].str.contains('^test$', flags=re.I, regex=True) |

        zoho_df['First Name'].str.contains('test 1', flags=re.I, regex=True) |
        zoho_df['Last Name'].str.contains('test 2', flags=re.I, regex=True)|
        zoho_df['First Name'].str.contains('test1', flags=re.I, regex=True) |
        zoho_df['Last Name'].str.contains('test2', flags=re.I, regex=True)|

        zoho_df['First Name'].str.contains(' test ', flags=re.I, regex=True) | 
        zoho_df['Last Name'].str.contains(' test ', flags=re.I, regex=True) | 
        zoho_df['Last Name'].str.contains(' test$', flags=re.I, regex=True)
    ]

count_leads_test = test_leads.shape[0]

# test_leads.to_excel(r'C:\Users\fajar.fatoni\Documents\Python\Data\test_leads_2.xlsx', index=False)
test_leads

,Resort Name,Departure_Date,Month Year,Booking Id,,Booker Title,Booker First Name,Booker Last Name,Email,TelNo,Concat_Tel_Mob,Media_Group,Source,MarketingOptIn,Market_Segment,Arrival Status,Contact Type


In [10]:
zoho_df.drop(test_leads.index, inplace=True)
zoho_df.reset_index(drop=True, inplace=True)

count_after_dedup_test = zoho_df.shape[0]

In [11]:
# dnm_leads = zoho_df.loc[(zoho_df['Do Not Mail'] == 'true') | (zoho_df['Do Not Contact'] == 'true')]
# count_dnm = dnm_leads.shape[0]

In [12]:
# zoho_df.drop(dnm_leads.index, inplace=True)
# zoho_df.reset_index(drop=True, inplace=True)

# count_after_dedup_dnm = zoho_df.shape[0]

In [13]:
# Cek Email OTA
email_OTA = m.find_OTA(zoho_df, 'Email')
count_email_OTA = email_OTA.shape[0]
# email_OTA.to_excel(r'C:\Users\fajar.fatoni\Documents\Python\Data\email_OTA.xlsx', index=False)

email_OTA

,Resort Name,Departure_Date,Month Year,Booking Id,,Booker Title,Booker First Name,Booker Last Name,Email,TelNo,Concat_Tel_Mob,Media_Group,Source,MarketingOptIn,Market_Segment,Arrival Status,Contact Type


In [14]:
# delete email OTA
count_after_clean_OTA = m.delete_OTA(zoho_df, 'Email')
# zoho_df.to_excel(r'C:\Users\fajar.fatoni\Documents\Python\Data\after_clean_OTA.xlsx', index=False)

zoho_df

,Resort Name,Departure_Date,Month Year,Booking Id,,Booker Title,Booker First Name,Booker Last Name,Email,TelNo,Concat_Tel_Mob,Media_Group,Source,MarketingOptIn,Market_Segment,Arrival Status,Contact Type
0,Karma Salford Hall,29/12/2024,Desember 2024,KSH019860,,Mrs,Alex And,Jack,alexandjack@gmail.com,empty,empty,GROUP,,Yes,COMP,Arrival Completed,Email and Phone
1,Karma Salford Hall,26/12/2024,Desember 2024,KSH020971,,Mrs,Erika,Goodman,egoodman96@gmail.com,66988930191,7973656599,RETAIL,,Yes,OTA,Arrival Completed,Email and Phone
2,Karma Salford Hall,18/12/2024,Desember 2024,KSH020039,,Mr,John,Haw,john.haw@fidelity-energy.co.uk,447747635540,447747635540,RETAIL,,Yes,DIR,Arrival Completed,Email and Phone
3,Karma Salford Hall,18/12/2024,Desember 2024,KSH020031,,Mr,Steven,Hawes,steveh@salomeurope.com,7918800410,7918800410,DIGITAL,,Yes,DIR,Arrival Completed,Email and Phone
4,Karma Salford Hall,11/12/2024,Desember 2024,KSH021368,,Mr,Tim,Brent,brenttjl@gmail.com,447789768665,447789768665,RETAIL,,Yes,OTA,Arrival Completed,Email and Phone
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
536,Karma Salford Hall,01/06/2025,Juni 2025,,,,Dianna-lee,,dianna.duncan@hotmail.com,61435938676,empty,,,Yes,EVENT,Arrival Completed,Email and Phone
537,Karma Salford Hall,01/06/2025,Juni 2025,,,,Rebecca,Carter,becky.smart.bs@googlemail.com,447969059458,empty,,,Yes,EVENT,Arrival Completed,Email and Phone
538,Karma Salford Hall,01/06/2025,Juni 2025,,,,Dilesh,parmar,dilesh.parmar@live.co.uk,447858224443,empty,,,Yes,EVENT,Arrival Completed,Email and Phone
539,Karma Salford Hall,01/06/2025,Juni 2025,,,,Louis,Radtke,lousterradtke@gmail.com,447796148410,empty,,,Yes,EVENT,Arrival Completed,Email and Phone


In [15]:
# Cek Email Karma
email_karma = m.find_karma(zoho_df, 'Email')
count_email_karma = email_karma.shape[0]
# email_OTA.to_excel(r'C:\Users\fajar.fatoni\Documents\Python\Data\email_OTA.xlsx', index=False)

email_karma

,Resort Name,Departure_Date,Month Year,Booking Id,,Booker Title,Booker First Name,Booker Last Name,Email,TelNo,Concat_Tel_Mob,Media_Group,Source,MarketingOptIn,Market_Segment,Arrival Status,Contact Type


In [16]:
# delete email Karma
count_after_clean_karma = m.delete_karma(zoho_df, 'Email')
# zoho_df.to_excel(r'C:\Users\fajar.fatoni\Documents\Python\Data\after_clean_OTA.xlsx', index=False)

zoho_df

,Resort Name,Departure_Date,Month Year,Booking Id,,Booker Title,Booker First Name,Booker Last Name,Email,TelNo,Concat_Tel_Mob,Media_Group,Source,MarketingOptIn,Market_Segment,Arrival Status,Contact Type
0,Karma Salford Hall,29/12/2024,Desember 2024,KSH019860,,Mrs,Alex And,Jack,alexandjack@gmail.com,empty,empty,GROUP,,Yes,COMP,Arrival Completed,Email and Phone
1,Karma Salford Hall,26/12/2024,Desember 2024,KSH020971,,Mrs,Erika,Goodman,egoodman96@gmail.com,66988930191,7973656599,RETAIL,,Yes,OTA,Arrival Completed,Email and Phone
2,Karma Salford Hall,18/12/2024,Desember 2024,KSH020039,,Mr,John,Haw,john.haw@fidelity-energy.co.uk,447747635540,447747635540,RETAIL,,Yes,DIR,Arrival Completed,Email and Phone
3,Karma Salford Hall,18/12/2024,Desember 2024,KSH020031,,Mr,Steven,Hawes,steveh@salomeurope.com,7918800410,7918800410,DIGITAL,,Yes,DIR,Arrival Completed,Email and Phone
4,Karma Salford Hall,11/12/2024,Desember 2024,KSH021368,,Mr,Tim,Brent,brenttjl@gmail.com,447789768665,447789768665,RETAIL,,Yes,OTA,Arrival Completed,Email and Phone
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
536,Karma Salford Hall,01/06/2025,Juni 2025,,,,Dianna-lee,,dianna.duncan@hotmail.com,61435938676,empty,,,Yes,EVENT,Arrival Completed,Email and Phone
537,Karma Salford Hall,01/06/2025,Juni 2025,,,,Rebecca,Carter,becky.smart.bs@googlemail.com,447969059458,empty,,,Yes,EVENT,Arrival Completed,Email and Phone
538,Karma Salford Hall,01/06/2025,Juni 2025,,,,Dilesh,parmar,dilesh.parmar@live.co.uk,447858224443,empty,,,Yes,EVENT,Arrival Completed,Email and Phone
539,Karma Salford Hall,01/06/2025,Juni 2025,,,,Louis,Radtke,lousterradtke@gmail.com,447796148410,empty,,,Yes,EVENT,Arrival Completed,Email and Phone


In [ ]:
find_member_booking = zoho_df.copy()

find_member_booking.loc[find_member_booking['Phone'].isin(df_booking['Phone1']), 'Same Value Find'] = 'Booking-Phone1'
find_member_booking.loc[find_member_booking['Phone'].isin(df_booking['Mobile']), 'Same Value Find'] = 'Booking-Mobile'
find_member_booking.loc[find_member_booking['Mobile'].isin(df_booking['Phone1']), 'Same Value Find'] = 'Booking-Phone1' 
find_member_booking.loc[find_member_booking['Mobile'].isin(df_booking['Mobile']), 'Same Value Find'] = 'Booking-Mobile'
find_member_booking.loc[find_member_booking['Email'].isin(df_booking['Email']), 'Same Value Find'] = 'Booking-Email'

find_member_booking.loc[find_member_booking['Phone'].isin(df_phone['Mobile']), 'Same Value Find'] = 'Member-Mobile' 
find_member_booking.loc[find_member_booking['Mobile'].isin(df_phone['Mobile']), 'Same Value Find'] = 'Member-Mobile'

find_member_booking.loc[find_member_booking['Email'].isin(df_email['Email']), 'Same Value Find'] = 'Member-Email'

find_member_booking.drop(find_member_booking.loc[find_member_booking['Same Value Find'].isna()].index, inplace=True)

count_booking_dedup = find_member_booking.loc[find_member_booking['Same Value Find'].str.contains('Booking', flags=re.I, regex=True)].shape[0]
count_member_dedup = find_member_booking.loc[find_member_booking['Same Value Find'].str.contains('Member', flags=re.I, regex=True)].shape[0]

find_member_booking.replace('empty', '', inplace=True)
# find_member_booking.to_csv(r'C:\Users\fajar.fatoni\Documents\Python\Data\booking-member-dedup.csv', index=False)
# find_member_booking.to_excel(r'C:\Users\fajar.fatoni\Documents\Python\Data\booking-member-dedup.xlsx', index=False)

#find_member_booking.loc[find_member_booking['Same Value Find'] == 'Booking-Phone1'] 

C:\Users\fajar.fatoni\AppData\Local\Temp\ipykernel_15972\399918518.py:19: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  find_member_booking.replace('empty', '', inplace=True)


In [ ]:
# data phone, mobile dam email member yang ada di master data 
count_wash_member = zoho_df.loc[
    zoho_df['Phone'].isin(df_booking['Phone1']) | 
    zoho_df['Phone'].isin(df_booking['Mobile']) |
    zoho_df['Mobile'].isin(df_booking['Phone1']) | 
    zoho_df['Mobile'].isin(df_booking['Mobile']) |
    zoho_df['Email'].isin(df_booking['Email'])| 

    zoho_df['Phone'].isin(df_phone['Mobile']) | 
    zoho_df['Mobile'].isin(df_phone['Mobile']) |

    zoho_df['Email'].isin(df_email['Email'])

].shape[0]

zoho_df.drop(zoho_df.loc[
    zoho_df['Phone'].isin(df_booking['Phone1']) | 
    zoho_df['Phone'].isin(df_booking['Mobile']) |
    zoho_df['Mobile'].isin(df_booking['Phone1']) | 
    zoho_df['Mobile'].isin(df_booking['Mobile']) |
    zoho_df['Email'].isin(df_booking['Email']) |
    
    zoho_df['Phone'].isin(df_phone['Mobile']) | 
    zoho_df['Mobile'].isin(df_phone['Mobile']) |

    zoho_df['Email'].isin(df_email['Email'])

    ].index, inplace=True)
    
zoho_df.reset_index(drop=True, inplace=True)

count_after_wash_member = zoho_df.shape[0]


In [ ]:
count_email_only = zoho_df.loc[(zoho_df['Email'] != 'empty') & ((zoho_df['Phone'] == 'empty') & (zoho_df['Phone'] == 'empty'))].shape[0]
count_phone_only = zoho_df.loc[(zoho_df['Email'] == 'empty') & ((zoho_df['Phone'] != 'empty') | (zoho_df['Phone'] != 'empty'))].shape[0]
count_email_and_phone = zoho_df.loc[(zoho_df['Email'] != 'empty') & ((zoho_df['Phone'] != 'empty') | (zoho_df['Phone'] != 'empty'))].shape[0]

count_blank = zoho_df.loc[(zoho_df['Email'] =='empty') & (zoho_df['Phone']=='empty') & (zoho_df['Mobile']=='empty')].shape[0]

# zoho_df.loc[(zoho_df['Email'] =='empty') & (zoho_df['Phone']=='empty') & (zoho_df['Mobile']=='empty')].to_excel(r'C:\Users\fajar.fatoni\Documents\Python\Data\blank_data.xlsx', index=False)

zoho_df.loc[(zoho_df['Email'] != 'empty') & ((zoho_df['Phone'] == 'empty') & (zoho_df['Mobile'] == 'empty')), 'Contact Type'] = 'Email Only'
zoho_df.loc[(zoho_df['Email'] == 'empty') & ((zoho_df['Phone'] != 'empty') | (zoho_df['Mobile'] != 'empty')), 'Contact Type'] = 'Phone Only'
zoho_df.loc[(zoho_df['Email'] != 'empty') & ((zoho_df['Phone'] != 'empty') | (zoho_df['Mobile'] != 'empty')), 'Contact Type'] = 'Email and Phone'

zoho_df.drop(zoho_df.loc[(zoho_df['Email'] =='empty') & (zoho_df['Phone']=='empty') & (zoho_df['Mobile']=='empty')].index, inplace=True)
zoho_df.reset_index(drop=True, inplace=True)

count_after_wash_blank = zoho_df.shape[0]

In [20]:
arr_dedup = [count_deleted_wrong_format, count_duplicate_general, count_leads_test, count_email_OTA, count_email_karma, count_blank]
count_sum_dedup = sum(arr_dedup)
count_sum_dedup

7

In [21]:
print("==========================================================")
print("Summary")
print("==========================================================")
print("Jumlah Filter Leads Awal : "+str(count_filter_awal))
print("Jumlah Format Email Salah : "+str(count_deleted_wrong_format))
print("Jumlah Setelah Clean Format Email Salah : "+str(count_after_deleted_wrong_format))
print("Jumlah Duplicate (Phone, Mobile dan Email) : "+str(count_duplicate_general))
print("Jumlah Setelah Clean Duplicate : "+str(count_after_dedup_duplicate))
print("Jumlah Lead Test : "+str(count_leads_test))
print("Jumlah Setelah Clean Lead Test : "+str(count_after_dedup_test))
# print("Jumlah Lead Do Not Mail : "+str(count_dnm))
# print("Jumlah Setelah Clean Lead Do Not Mail : "+str(count_after_dedup_dnm))
print("Jumlah Email OTA : "+str(count_email_OTA))
print("Jumlah Setelah Clean Email OTA : "+str(count_after_clean_OTA))
print("Jumlah Email Karma : "+str(count_email_karma))
print("Jumlah Setelah Clean Email Karma : "+str(count_after_clean_karma))
print("Jumlah Wash Member : "+str(count_member_dedup))
print("Jumlah Wash Booking : "+str(count_booking_dedup))
print("Jumlah Setelah Wash Member : "+str(count_after_wash_member))
print("Jumlah Phone Email Blank (Setelah phone yang tidak valid dihapus) : "+str(count_blank))
print("Jumlah Setelah Wash Phone Email Blank : "+str(count_after_wash_blank))
print("==========================================================")
print("Contact Type")
print("==========================================================")
print("Jumlah Email Only : "+str(count_email_only))
print("Jumlah Phone Only : "+str(count_phone_only))
print("Jumlah Email dan Phone : "+str(count_email_and_phone))

Summary
Jumlah Filter Leads Awal : 548
Jumlah Format Email Salah : 7
Jumlah Setelah Clean Format Email Salah : 541
Jumlah Duplicate (Phone, Mobile dan Email) : 0
Jumlah Setelah Clean Duplicate : 541
Jumlah Lead Test : 0
Jumlah Setelah Clean Lead Test : 541
Jumlah Email OTA : 0
Jumlah Setelah Clean Email OTA : 541
Jumlah Email Karma : 0
Jumlah Setelah Clean Email Karma : 541
Jumlah Wash Member : 2
Jumlah Wash Booking : 0
Jumlah Setelah Wash Member : 539
Jumlah Phone Email Blank (Setelah phone yang tidak valid dihapus) : 0
Jumlah Setelah Wash Phone Email Blank : 539
Contact Type
Jumlah Email Only : 196
Jumlah Phone Only : 0
Jumlah Email dan Phone : 343


In [22]:
print("==========================================================")
print("Summary Report")
print("==========================================================")
print("Jumlah Data Awal : "+str(count_filter_awal))
print("Jumlah Wash Dedup : "+str(count_sum_dedup))
print("Jumlah Wash Member : "+str(count_member_dedup))
print("Jumlah Wash Booking : "+str(count_booking_dedup))
print("Jumlah Data Akhir : "+str(count_after_wash_blank))

Summary Report
Jumlah Data Awal : 548
Jumlah Wash Dedup : 7
Jumlah Wash Member : 2
Jumlah Wash Booking : 0
Jumlah Data Akhir : 539


In [ ]:
zoho_df.replace('empty', '', inplace=True, regex=True)

zoho_df['First Name'] = zoho_df['First Name'].str.title()
zoho_df['Last Name'] = zoho_df['Last Name'].str.title()

# zoho_df.to_csv(r'C:\Users\fajar.fatoni\Documents\Python\Data\KLOM.csv', index=False)
# zoho_df.to_excel(r'C:\Users\fajar.fatoni\Documents\Python\Data\newsletter-clean.xlsx', index=False)

In [25]:
display(zoho_df)
# Update keseluruhan data, delete data yang lama terlebih dahulu
# clear all row before update
worksheet.clear()
worksheet.update([zoho_df.columns.values.tolist()] + zoho_df.values.tolist())

,Resort Name,Departure_Date,Month Year,Booking Id,,Booker Title,Booker First Name,Booker Last Name,Email,TelNo,Concat_Tel_Mob,Media_Group,Source,MarketingOptIn,Market_Segment,Arrival Status,Contact Type
0,Karma Salford Hall,29/12/2024,Desember 2024,KSH019860,,Mrs,Alex And,Jack,alexandjack@gmail.com,,,GROUP,,Yes,COMP,Arrival Completed,Email Only
1,Karma Salford Hall,26/12/2024,Desember 2024,KSH020971,,Mrs,Erika,Goodman,egoodman96@gmail.com,66988930191,7973656599,RETAIL,,Yes,OTA,Arrival Completed,Email and Phone
2,Karma Salford Hall,18/12/2024,Desember 2024,KSH020039,,Mr,John,Haw,john.haw@fidelity-energy.co.uk,447747635540,447747635540,RETAIL,,Yes,DIR,Arrival Completed,Email and Phone
3,Karma Salford Hall,18/12/2024,Desember 2024,KSH020031,,Mr,Steven,Hawes,steveh@salomeurope.com,7918800410,7918800410,DIGITAL,,Yes,DIR,Arrival Completed,Email and Phone
4,Karma Salford Hall,11/12/2024,Desember 2024,KSH021368,,Mr,Tim,Brent,brenttjl@gmail.com,447789768665,447789768665,RETAIL,,Yes,OTA,Arrival Completed,Email and Phone
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
534,Karma Salford Hall,01/06/2025,Juni 2025,,,,Jackie,Tandy,jackie.roche@talk21.com,447941544298,,,,Yes,EVENT,Arrival Completed,Email and Phone
535,Karma Salford Hall,01/06/2025,Juni 2025,,,,Rebecca,Carter,becky.smart.bs@googlemail.com,447969059458,,,,Yes,EVENT,Arrival Completed,Email and Phone
536,Karma Salford Hall,01/06/2025,Juni 2025,,,,Dilesh,Parmar,dilesh.parmar@live.co.uk,447858224443,,,,Yes,EVENT,Arrival Completed,Email and Phone
537,Karma Salford Hall,01/06/2025,Juni 2025,,,,Louis,Radtke,lousterradtke@gmail.com,447796148410,,,,Yes,EVENT,Arrival Completed,Email and Phone


{'spreadsheetId': '1ZmyQRySSr6TTqEzOFdhf_U6VEaYwG3Uolz_nwJBoWN8',
 'updatedRange': 'KSH!A1:Q540',
 'updatedRows': 540,
 'updatedColumns': 17,
 'updatedCells': 9180}